In [1]:
import sys
import pandas as pd
import numpy as np
import IPython
from IPython.display import display, HTML, Latex

print( f"Python {sys.version}" )
print( f"Pandas {pd.__version__}" )
print( f"NumPy {np.__version__}" )
print( f"IPython {IPython.__version__}" )

Python 3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0]
Pandas 2.1.4
NumPy 1.26.4
IPython 7.34.0


In [2]:
# Load our same old pokemon dataset
pokemon_df = pd.read_csv('https://raw.githubusercontent.com/ShaileshDhama/Exploratory-Data-Analysis-On-Pokemon-Dataset/master/Complete%20Pokemon.csv')
pokemon_df.head()

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0


# 1. Pandas: Combine columns with database-style join operations

Combine separate datasets to unify and better understand the data

1. Pandas' `join()`
  - Similar to SQL JOIN, but <u>primarily used for joining based on the row label</u>
2. Pandas' `merge()`
  - Similar to SQL JOIN, and is more flexible than `join()`
  - `merge()` <u>allows joining on one or more columns or the row label</u>. You can specify which columns you want to join on, making it closer to SQL-style joins.


## 1.1 Prepare two DataFrames

In [3]:
# Create the subset of our same old pokemon DataFrame to play around
df1_pokemon = pokemon_df.loc[ : , ['pokedex_number','name','type2'] ]   # Choose columns

df1_pokemon.info()
df1_pokemon

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pokedex_number  801 non-null    int64 
 1   name            801 non-null    object
 2   type2           417 non-null    object
dtypes: int64(1), object(2)
memory usage: 18.9+ KB


,pokedex_number,name,type2
0,1,Bulbasaur,poison
1,2,Ivysaur,poison
2,3,Venusaur,poison
3,4,Charmander,NaN
4,5,Charmeleon,NaN
...,...,...,...
796,797,Celesteela,flying
797,798,Kartana,steel
798,799,Guzzlord,dragon
799,800,Necrozma,NaN


In [4]:
# Upload another pokemon dataset from the given csv file
# Credit: https://www.kaggle.com/datasets/n2cholas/competitive-pokemon-dataset
df2_move = pd.read_csv('pokemon-move-data.csv')
df2_move

,Index,Name,Type,Category,Contest,PP,Power,Accuracy,Generation
0,1,Pound,Normal,Physical,Tough,35,40.0,100.0,1
1,2,Karate Chop,Fighting,Physical,Tough,25,50.0,100.0,1
2,3,Double Slap,Normal,Physical,Cute,10,15.0,85.0,1
3,4,Comet Punch,Normal,Physical,Tough,15,18.0,85.0,1
4,5,Mega Punch,Normal,Physical,Tough,20,80.0,85.0,1
...,...,...,...,...,...,...,...,...,...
723,724,Searing Sunraze Smash,Steel,Special,???,1,200.0,NaN,7
724,725,Menacing Moonraze Maelstrom,Ghost,Special,???,1,200.0,NaN,7
725,726,Let's Snuggle Forever,Fairy,Physical,???,1,190.0,NaN,7
726,727,Splintered Stormshards,Rock,Physical,???,1,190.0,NaN,7


In [5]:
# Convert the 'Type' attribute of df2_move to lowercase string to match with the 'type2' attribute in df1_pokemon
df2_move['Type'] = df2_move['Type'].str.lower()
df2_move

,Index,Name,Type,Category,Contest,PP,Power,Accuracy,Generation
0,1,Pound,normal,Physical,Tough,35,40.0,100.0,1
1,2,Karate Chop,fighting,Physical,Tough,25,50.0,100.0,1
2,3,Double Slap,normal,Physical,Cute,10,15.0,85.0,1
3,4,Comet Punch,normal,Physical,Tough,15,18.0,85.0,1
4,5,Mega Punch,normal,Physical,Tough,20,80.0,85.0,1
...,...,...,...,...,...,...,...,...,...
723,724,Searing Sunraze Smash,steel,Special,???,1,200.0,NaN,7
724,725,Menacing Moonraze Maelstrom,ghost,Special,???,1,200.0,NaN,7
725,726,Let's Snuggle Forever,fairy,Physical,???,1,190.0,NaN,7
726,727,Splintered Stormshards,rock,Physical,???,1,190.0,NaN,7


Before continue combining the two datasets, let's first check values in both `df1_pokemon['type2']` and `df2_move['Type']`:

In [6]:
print( '=== Type2: df1_pokemon ===' )
print( df1_pokemon['type2'].nunique(dropna=False) )
print( df1_pokemon['type2'].unique() )  # Apply sorted() to this result produces an error as float (nan) cannot be compared with str

print( '\n=== Type: df2_move ===' )
print( df2_move['Type'].nunique(dropna=False) )
print( sorted( df2_move['Type'].unique() ) )

print( '\n=== Type: intersection ===' )
lst = [ t for t in df1_pokemon['type2'].unique() if t in df2_move['Type'].unique() ]
print( len(lst) )
print( sorted(lst) )

=== Type2: df1_pokemon ===
19
['poison' nan 'flying' 'dark' 'electric' 'ice' 'ground' 'fairy' 'grass'
 'fighting' 'psychic' 'steel' 'fire' 'rock' 'water' 'dragon' 'ghost' 'bug'
 'normal']

=== Type: df2_move ===
18
['bug', 'dark', 'dragon', 'electric', 'fairy', 'fighting', 'fire', 'flying', 'ghost', 'grass', 'ground', 'ice', 'normal', 'poison', 'psychic', 'rock', 'steel', 'water']

=== Type: intersection ===
18
['bug', 'dark', 'dragon', 'electric', 'fairy', 'fighting', 'fire', 'flying', 'ghost', 'grass', 'ground', 'ice', 'normal', 'poison', 'psychic', 'rock', 'steel', 'water']


## 1.2 Combine columns based on common values

`merge()` is useful when we want to combine rows that share data. Use `merge()` when functionality similar to a database’s `join` operations is required.

### 1.2.1 `merge()` with a single key

#### Inner join:

In [7]:
# Default is how='inner'
# If the two columns to join on share the same label ('col_name'), we can use pd.merge( df1, df2, on='col_name' ) instead

# Alternative 1
df_inner = pd.merge( df1_pokemon, df2_move, left_on='type2', right_on='Type' )

# Alternative 2
#df_inner = df1_pokemon.merge( df2_move, left_on='type2', right_on='Type' )

# Display results
df_inner

,pokedex_number,name,type2,Index,Name,Type,Category,Contest,PP,Power,Accuracy,Generation
0,1,Bulbasaur,poison,40,Poison Sting,poison,Physical,Clever,35,15.0,100.0,1
1,1,Bulbasaur,poison,51,Acid,poison,Special,Clever,30,40.0,100.0,1
2,1,Bulbasaur,poison,77,Poison Powder,poison,Status,Clever,35,NaN,75.0,1
3,1,Bulbasaur,poison,92,Toxic,poison,Status,Clever,10,NaN,NaN,1
4,1,Bulbasaur,poison,123,Smog,poison,Special,Tough,20,30.0,70.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
13915,695,Heliolisk,normal,686,Revelation Dance,normal,Special,???,15,90.0,100.0,7
13916,695,Heliolisk,normal,701,Pulverizing Pancake,normal,Physical,???,1,210.0,NaN,7
13917,695,Heliolisk,normal,702,Extreme Evoboost,normal,Status,???,1,NaN,NaN,7
13918,695,Heliolisk,normal,715,Tearful Look,normal,Status,???,20,NaN,NaN,7


In [8]:
# Inspect the new DataFrame regarding one selected pokemon
df_inner[ df_inner['pokedex_number'] == 1 ]

,pokedex_number,name,type2,Index,Name,Type,Category,Contest,PP,Power,Accuracy,Generation
0,1,Bulbasaur,poison,40,Poison Sting,poison,Physical,Clever,35,15.0,100.0,1
1,1,Bulbasaur,poison,51,Acid,poison,Special,Clever,30,40.0,100.0,1
2,1,Bulbasaur,poison,77,Poison Powder,poison,Status,Clever,35,NaN,75.0,1
3,1,Bulbasaur,poison,92,Toxic,poison,Status,Clever,10,NaN,NaN,1
4,1,Bulbasaur,poison,123,Smog,poison,Special,Tough,20,30.0,70.0,1
5,1,Bulbasaur,poison,124,Sludge,poison,Special,Tough,20,65.0,100.0,1
6,1,Bulbasaur,poison,139,Poison Gas,poison,Status,Clever,40,NaN,NaN,1
7,1,Bulbasaur,poison,151,Acid Armor,poison,Status,Tough,20,NaN,NaN,1
8,1,Bulbasaur,poison,188,Sludge Bomb,poison,Special,Tough,10,90.0,100.0,2
9,1,Bulbasaur,poison,305,Poison Fang,poison,Physical,Clever,15,50.0,100.0,3


In [9]:
# Inspect the new DataFrame whether there are any unmatched rows (rows where 'type2' is NA)
df_inner.isna().sum()

,0
pokedex_number,0
name,0
type2,0
Index,0
Name,0
Type,0
Category,0
Contest,0
PP,0
Power,5509


#### Left join (left outer join):

The new merged DataFrame <u>includes all rows from the left DataFrame</u>, while discarding rows from the right DataFrame that don’t have a match in the key column of the left DataFrame.

In [10]:
# Alternative 1
df_left = pd.merge( df1_pokemon, df2_move, left_on='type2', right_on='Type', how='left' )

# Alternative 2
#df_left = df1_pokemon.merge( df2_move, left_on='type2', right_on='Type', how='left' )

# Display
df_left

,pokedex_number,name,type2,Index,Name,Type,Category,Contest,PP,Power,Accuracy,Generation
0,1,Bulbasaur,poison,40.0,Poison Sting,poison,Physical,Clever,35.0,15.0,100.0,1.0
1,1,Bulbasaur,poison,51.0,Acid,poison,Special,Clever,30.0,40.0,100.0,1.0
2,1,Bulbasaur,poison,77.0,Poison Powder,poison,Status,Clever,35.0,NaN,75.0,1.0
3,1,Bulbasaur,poison,92.0,Toxic,poison,Status,Clever,10.0,NaN,NaN,1.0
4,1,Bulbasaur,poison,123.0,Smog,poison,Special,Tough,20.0,30.0,70.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14299,801,Magearna,fairy,666.0,Floral Healing,fairy,Status,???,10.0,NaN,NaN,7.0
14300,801,Magearna,fairy,698.0,Guardian of Alola,fairy,Special,???,1.0,NaN,NaN,7.0
14301,801,Magearna,fairy,705.0,Fleur Cannon,fairy,Special,???,5.0,130.0,90.0,7.0
14302,801,Magearna,fairy,717.0,Nature's Madness,fairy,Special,???,10.0,NaN,90.0,7.0


In [11]:
# Inspect the new DataFrame whether there are any unmatched rows (rows where 'type2' is NA)
print( df_left.isna().sum() )
display( df_left[ df_left['type2'].isna() ] )

pokedex_number       0
name                 0
type2              384
Index              384
Name               384
Type               384
Category           384
Contest            384
PP                 384
Power             5893
Accuracy          5440
Generation         384
dtype: int64


,pokedex_number,name,type2,Index,Name,Type,Category,Contest,PP,Power,Accuracy,Generation
84,4,Charmander,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,5,Charmeleon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,7,Squirtle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,8,Wartortle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,9,Blastoise,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13850,782,Jangmo-o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14035,789,Cosmog,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14036,790,Cosmoem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14206,796,Xurkitree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Right join (right outer join):

<u>All rows from the right DataFrame will be retained</u>, while rows in the left DataFrame without a match in the key column of the right DataFrame will be discarded.

#### Outer join (full outer join):

<u>All rows from both DataFrames will be present in the new DataFrame</u> as this outer join returns all rows which either have a match in the left or right DataFrames.

In [12]:
# Alternative 1
df_outer = pd.merge( df1_pokemon, df2_move, left_on='type2', right_on='Type', how='outer' )

# Alternative 2
#df_outer = df1_pokemon.merge( df2_move, left_on='type2', right_on='Type', how='outer' )

# Display
df_outer

,pokedex_number,name,type2,Index,Name,Type,Category,Contest,PP,Power,Accuracy,Generation
0,1,Bulbasaur,poison,40.0,Poison Sting,poison,Physical,Clever,35.0,15.0,100.0,1.0
1,1,Bulbasaur,poison,51.0,Acid,poison,Special,Clever,30.0,40.0,100.0,1.0
2,1,Bulbasaur,poison,77.0,Poison Powder,poison,Status,Clever,35.0,NaN,75.0,1.0
3,1,Bulbasaur,poison,92.0,Toxic,poison,Status,Clever,10.0,NaN,NaN,1.0
4,1,Bulbasaur,poison,123.0,Smog,poison,Special,Tough,20.0,30.0,70.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14299,695,Heliolisk,normal,686.0,Revelation Dance,normal,Special,???,15.0,90.0,100.0,7.0
14300,695,Heliolisk,normal,701.0,Pulverizing Pancake,normal,Physical,???,1.0,210.0,NaN,7.0
14301,695,Heliolisk,normal,702.0,Extreme Evoboost,normal,Status,???,1.0,NaN,NaN,7.0
14302,695,Heliolisk,normal,715.0,Tearful Look,normal,Status,???,20.0,NaN,NaN,7.0


#### Cross join (cartesian product):

Create the cartesian product from both DataFrames while preserving the order of the left keys

In [13]:
# Alternative 1
df_cross = pd.merge( df1_pokemon, df2_move, how='cross' )

# Alternative 2
#df_cross = df1_pokemon.merge( df2_move, how='cross' )

# Display
df_cross

,pokedex_number,name,type2,Index,Name,Type,Category,Contest,PP,Power,Accuracy,Generation
0,1,Bulbasaur,poison,1,Pound,normal,Physical,Tough,35,40.0,100.0,1
1,1,Bulbasaur,poison,2,Karate Chop,fighting,Physical,Tough,25,50.0,100.0,1
2,1,Bulbasaur,poison,3,Double Slap,normal,Physical,Cute,10,15.0,85.0,1
3,1,Bulbasaur,poison,4,Comet Punch,normal,Physical,Tough,15,18.0,85.0,1
4,1,Bulbasaur,poison,5,Mega Punch,normal,Physical,Tough,20,80.0,85.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
583123,801,Magearna,fairy,724,Searing Sunraze Smash,steel,Special,???,1,200.0,NaN,7
583124,801,Magearna,fairy,725,Menacing Moonraze Maelstrom,ghost,Special,???,1,200.0,NaN,7
583125,801,Magearna,fairy,726,Let's Snuggle Forever,fairy,Physical,???,1,190.0,NaN,7
583126,801,Magearna,fairy,727,Splintered Stormshards,rock,Physical,???,1,190.0,NaN,7


### 1.2.2 `merge()` with multiple keys

In [14]:
# Create dummy DataFrames
df1 = pd.DataFrame({'StudentID': [1, 2, 3, 4],
                    'CourseID': [101, 102, 103, 104],
                    'Grade': ['A', 'B', 'B', 'C']})

df2 = pd.DataFrame({'StudentID': [2, 3, 4, 5],
                    'CourseID': [102, 103, 104, 105],
                    'CourseName': ['Math', 'Science', 'History', 'English']})

display(df1)
display(df2)

,StudentID,CourseID,Grade
0,1,101,A
1,2,102,B
2,3,103,B
3,4,104,C


,StudentID,CourseID,CourseName
0,2,102,Math
1,3,103,Science
2,4,104,History
3,5,105,English


In [15]:
# Merge on multiple keys: Inner join

# Alternative 1
pd.merge(df1, df2, on=['StudentID', 'CourseID'], how='inner')

# Alternative 2
#pd.merge(df1, df2, left_on=['StudentID', 'CourseID'], right_on=['StudentID', 'CourseID'], how='inner')

,StudentID,CourseID,Grade,CourseName
0,2,102,B,Math
1,3,103,B,Science
2,4,104,C,History


In [16]:
# Merge on multiple keys: Full outer join

# Alternative 1
pd.merge(df1, df2, on=['StudentID', 'CourseID'], how='outer')

# Alternative 2
#pd.merge(df1, df2, left_on=['StudentID', 'CourseID'], right_on=['StudentID', 'CourseID'], how='outer')

,StudentID,CourseID,Grade,CourseName
0,1,101,A,NaN
1,2,102,B,Math
2,3,103,B,Science
3,4,104,C,History
4,5,105,NaN,English


## 1.3 Combine columns based on common indices (row labels)

Prepare the first DataFrame:

In [17]:
# Group and aggregate data
group1 = pokemon_df[ ['name', 'type2', 'hp', 'attack', 'defense' ] ] \
                        .groupby('type2', dropna=False)[['attack','defense']] \
                        .agg(['min','mean','max'])

print(f"{group1.shape=}")
print(f"{group1.columns=}")
group1

group1.shape=(19, 6)
group1.columns=MultiIndex([( 'attack',  'min'),
            ( 'attack', 'mean'),
            ( 'attack',  'max'),
            ('defense',  'min'),
            ('defense', 'mean'),
            ('defense',  'max')],
           )


attack                  defense                 
            min        mean  max     min        mean  max
type2                                                    
bug          40   76.000000  125      50   76.800000  100
dark         35   95.904762  164      20   67.238095  150
dragon       30   88.470588  121      35   84.058824  135
electric     38   74.777778  150      38   71.666667  120
fairy        20   63.482759  160      15   70.931034  150
fighting     61  107.960000  185      37   83.280000  139
fire         30   66.000000  120      30   73.846154  110
flying       20   78.389474  180      30   66.778947  140
ghost        40   95.571429  160      45   80.071429  150
grass        30   78.250000  131      30   75.250000  122
ground       45   86.058824  170      30   84.735294  230
ice          41   88.666667  132      40   81.933333  180
normal       38   52.750000   68      33   53.750000   72
poison       25   67.617647  150      30   59.529412  123
psychic      24   76.586207  145      15   84.310345  180
rock         10   86.857143  140      85  131.142857  230
steel        35   97.454545  181      60  106.272727  168
water        30   70.176471  125      32   86.941176  152
NaN           5   74.231771  180       5   67.955729  200

In [18]:
# Convert the 2-level column labels to the 1-label column labels
group1.columns = group1.columns.to_flat_index().str.join('_')

print(f"{group1.shape=}")
print(f"{group1.columns=}")
group1

group1.shape=(19, 6)
group1.columns=Index(['attack_min', 'attack_mean', 'attack_max', 'defense_min',
       'defense_mean', 'defense_max'],
      dtype='object')


,attack_min,attack_mean,attack_max,defense_min,defense_mean,defense_max
type2,,,,,,
bug,40,76.000000,125,50,76.800000,100
dark,35,95.904762,164,20,67.238095,150
dragon,30,88.470588,121,35,84.058824,135
electric,38,74.777778,150,38,71.666667,120
fairy,20,63.482759,160,15,70.931034,150
fighting,61,107.960000,185,37,83.280000,139
fire,30,66.000000,120,30,73.846154,110
flying,20,78.389474,180,30,66.778947,140
ghost,40,95.571429,160,45,80.071429,150


Prepare the second DataFrame:

In [19]:
# Group > count the number of moves in each group > rename the counting column
group2 = df2_move[['Type','Name']].groupby('Type').count()
group2 = group2.rename({'Name':'move_count'}, axis=1)

print(f"{group2.shape=}")
group2

group2.shape=(18, 1)


,move_count
Type,
bug,30
dark,39
dragon,19
electric,34
fairy,24
fighting,44
fire,36
flying,27
ghost,27


Merge two DataFrames based on common indices (row labels):

In [20]:
# Inner join

# Alternative 1: join() is a convenience method for joining columns based on (row) indices
print('-------------- join() --------------')
result1_inner = group1.join(group2, how='inner')      # default:how='left'
print(f"{result1_inner.shape=}")
display( result1_inner )

# Alternative 2: merge() can join with indices too but we need to set some more parameters explicitly
print('\n-------------- merge() --------------')
result2_inner = group1.merge(group2, how='inner', left_index=True, right_index=True)  # default:how='inner', left_index=False, right_index=False
print(f"{result2_inner.shape=}")
display( result2_inner )

-------------- join() --------------
result1_inner.shape=(18, 7)


,attack_min,attack_mean,attack_max,defense_min,defense_mean,defense_max,move_count
bug,40,76.000000,125,50,76.800000,100,30
dark,35,95.904762,164,20,67.238095,150,39
dragon,30,88.470588,121,35,84.058824,135,19
electric,38,74.777778,150,38,71.666667,120,34
fairy,20,63.482759,160,15,70.931034,150,24
fighting,61,107.960000,185,37,83.280000,139,44
fire,30,66.000000,120,30,73.846154,110,36
flying,20,78.389474,180,30,66.778947,140,27
ghost,40,95.571429,160,45,80.071429,150,27
grass,30,78.250000,131,30,75.250000,122,43



-------------- merge() --------------
result2_inner.shape=(18, 7)


,attack_min,attack_mean,attack_max,defense_min,defense_mean,defense_max,move_count
bug,40,76.000000,125,50,76.800000,100,30
dark,35,95.904762,164,20,67.238095,150,39
dragon,30,88.470588,121,35,84.058824,135,19
electric,38,74.777778,150,38,71.666667,120,34
fairy,20,63.482759,160,15,70.931034,150,24
fighting,61,107.960000,185,37,83.280000,139,44
fire,30,66.000000,120,30,73.846154,110,36
flying,20,78.389474,180,30,66.778947,140,27
ghost,40,95.571429,160,45,80.071429,150,27
grass,30,78.250000,131,30,75.250000,122,43


In [21]:
# Compare the two DataFrames and show their differences (if any)
diff = result1_inner.compare(result2_inner)  # default:keep_shape=False

print(f"{diff.shape=}")
diff

diff.shape=(0, 0)


Empty DataFrame
Columns: []
Index: []

In [22]:
# Left join

# Alternative 1
print('-------------- join() --------------')
result1_left = group1.join(group2)      # default:how='left'
print(f"{result1_left.shape=}")
display( result1_left )

# Alternative 2
print('\n-------------- merge() --------------')
result2_left = group1.merge(group2, how='left', left_index=True, right_index=True)  # default:how='inner', left_index=False, right_index=False
print(f"{result2_left.shape=}")
display( result2_left )

-------------- join() --------------
result1_left.shape=(19, 7)


,attack_min,attack_mean,attack_max,defense_min,defense_mean,defense_max,move_count
type2,,,,,,,
bug,40,76.000000,125,50,76.800000,100,30.0
dark,35,95.904762,164,20,67.238095,150,39.0
dragon,30,88.470588,121,35,84.058824,135,19.0
electric,38,74.777778,150,38,71.666667,120,34.0
fairy,20,63.482759,160,15,70.931034,150,24.0
fighting,61,107.960000,185,37,83.280000,139,44.0
fire,30,66.000000,120,30,73.846154,110,36.0
flying,20,78.389474,180,30,66.778947,140,27.0
ghost,40,95.571429,160,45,80.071429,150,27.0



-------------- merge() --------------
result2_left.shape=(19, 7)


,attack_min,attack_mean,attack_max,defense_min,defense_mean,defense_max,move_count
type2,,,,,,,
bug,40,76.000000,125,50,76.800000,100,30.0
dark,35,95.904762,164,20,67.238095,150,39.0
dragon,30,88.470588,121,35,84.058824,135,19.0
electric,38,74.777778,150,38,71.666667,120,34.0
fairy,20,63.482759,160,15,70.931034,150,24.0
fighting,61,107.960000,185,37,83.280000,139,44.0
fire,30,66.000000,120,30,73.846154,110,36.0
flying,20,78.389474,180,30,66.778947,140,27.0
ghost,40,95.571429,160,45,80.071429,150,27.0


In [23]:
# Compare the two DataFrames and show their differences (if any)
diff = result1_left.compare(result2_left)  # default:keep_shape=False

print(f"{diff.shape=}")
diff

diff.shape=(0, 0)


type2


In [24]:
# Compare the results from inner join and left join

# compare() can only compare identically-labeled (both index and columns) DataFrame objects
diff = result1_inner.compare(result1_left)  # ValueError

ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects

In [25]:
# A small example when there are differences for compare() to show
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
df2 = pd.DataFrame({'A': [1, 2, 4], 'B': [4, 5, 7], 'C': [7, 5, 6]})

# Compare the DataFrames
print('----- df1 -----')
display(df1)
print('\n----- df2 -----')
display(df2)
print('\n----- compare() -----')
display(df1.compare(df2))

----- df1 -----


,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9



----- df2 -----


,A,B,C
0,1,4,7
1,2,5,5
2,4,7,6



----- compare() -----


A          B          C      
  self other self other self other
1  NaN   NaN  NaN   NaN  8.0   5.0
2  3.0   4.0  6.0   7.0  9.0   6.0

SQL JOIN vs. pandas `join()` and `merge()`
- In SQL, the JOIN operation works on columns by default (e.g., JOIN ON column_name).
- In pandas, `merge()` is the closest equivalent to SQL-style joins, as you can specify columns to join on.
- `join()` in pandas is mostly for joining based on the index (row label), while SQL joins are typically done on columns.

# 2. Pandas: Combine data by stitching them across rows or columns

- With `merge()` (and `join()`), we can expect the resulting dataset to have rows from the parent datasets mixed in together, <u>often based on some commonality</u>.

- With `concat()`, <u>our datasets are just stitched together along an axis</u> — either the row axis or column axis. It is a simpler way to combine datasets and is often used to form a single, larger set to do additional operations on.

**EX:** Concat along axis=0 (row)

Stack many DataFrames on one another

In [26]:
# Choose only water pokemons
df_water = pokemon_df[['pokedex_number','name','type1','type2','abilities']][ pokemon_df['type1']=='water' ]

df_water = df_water.reset_index(drop=True)
df_water

,pokedex_number,name,type1,type2,abilities
0,7,Squirtle,water,NaN,"['Torrent', 'Rain Dish']"
1,8,Wartortle,water,NaN,"['Torrent', 'Rain Dish']"
2,9,Blastoise,water,NaN,"['Torrent', 'Rain Dish']"
3,54,Psyduck,water,NaN,"['Damp', 'Cloud Nine', 'Swift Swim']"
4,55,Golduck,water,NaN,"['Damp', 'Cloud Nine', 'Swift Swim']"
...,...,...,...,...,...
109,751,Dewpider,water,bug,"['Water Bubble', 'Water Absorb']"
110,752,Araquanid,water,bug,"['Water Bubble', 'Water Absorb']"
111,771,Pyukumuku,water,NaN,"['Innards Out', 'Unaware']"
112,779,Bruxish,water,psychic,"['Dazzling', 'Strong Jaw', 'Wonder Skin ']"


In [27]:
# Choose only fire pokemons
df_fire = pokemon_df[['type1','type2','pokedex_number','name']][ pokemon_df['type1']=='fire' ]

df_fire = df_fire.reset_index(drop=True)
df_fire

,type1,type2,pokedex_number,name
0,fire,NaN,4,Charmander
1,fire,NaN,5,Charmeleon
2,fire,flying,6,Charizard
3,fire,ice,37,Vulpix
4,fire,ice,38,Ninetales
5,fire,NaN,58,Growlithe
6,fire,NaN,59,Arcanine
7,fire,NaN,77,Ponyta
8,fire,NaN,78,Rapidash
9,fire,NaN,126,Magmar


In [28]:
# Concat to get a new DataFrame with both water and fire pokemons
# With ignore_index=True, the command will clear the existing index and reset it in the result
df = pd.concat( [df_water, df_fire], ignore_index=True  )    # default:axis=0
df

,pokedex_number,name,type1,type2,abilities
0,7,Squirtle,water,NaN,"['Torrent', 'Rain Dish']"
1,8,Wartortle,water,NaN,"['Torrent', 'Rain Dish']"
2,9,Blastoise,water,NaN,"['Torrent', 'Rain Dish']"
3,54,Psyduck,water,NaN,"['Damp', 'Cloud Nine', 'Swift Swim']"
4,55,Golduck,water,NaN,"['Damp', 'Cloud Nine', 'Swift Swim']"
...,...,...,...,...,...
161,725,Litten,fire,NaN,NaN
162,726,Torracat,fire,NaN,NaN
163,727,Incineroar,fire,dark,NaN
164,741,Oricorio,fire,flying,NaN


# 3. Pandas: Reshape the data

A tabular dataset can be written in two different formats:
1. Wide format
  - A wide format contains <u>values that **do not repeat** in a single column</u>.
  - For displaying data, we typically use a wide data format. Most datasets that we encounter in the real world will also be recorded in a wide format because it’s easier for our brains to interpret.
2. Long format
  - A long format contains <u>values that **do repeat** in a single column</u>.
  - During data processing and analyzing, the data should be in a long format. Also many functions might specifically require long-format data (e.g., when we’re visualizing multiple variables in a plot using statistical software, we typically must convert our data to a long format in order for the software to create the plot).

![picture](https://www.statology.org/wp-content/uploads/2021/12/wideLong1-1-768x543.png)

Read more about how to use wide- and long-format tables for further visualization with seaborn in https://seaborn.pydata.org/tutorial/data_structure.html

## 3.1 Pivot table (from long to wide)



### **EX1:** Pivot without aggregation using `pivot()`

Codes from https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html

In [29]:
# Create a long DataFrame
df = pd.DataFrame({'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
                   'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'baz': [1, 2, 3, 4, 5, 6],
                   'zoo': ['x', 'y', 'z', 'q', 'w', 't']})
df

,foo,bar,baz,zoo
0,one,A,1,x
1,one,B,2,y
2,one,C,3,z
3,two,A,4,q
4,two,B,5,w
5,two,C,6,t


In [30]:
# pivot()
# - 'index' refers to the column whose unique values will be used as row labels
# - 'columns' refers to the column whose unique values will be used as column labels
# - 'values' refers to the column whose values will be used to fill in the DataFrame
df1 = df.pivot(index='foo', columns='bar', values='baz')

print('=== After pivot() ===')
display(df1)
print('Row:', df1.index)
print('Column:', df1.columns)

=== After pivot() ===


bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


Row: Index(['one', 'two'], dtype='object', name='foo')
Column: Index(['A', 'B', 'C'], dtype='object', name='bar')


In [31]:
# Reset index
print('\n=== After reset_index ===')
df1.reset_index(inplace=True)
display(df1)
print(df1.index)
print(df1.columns)


=== After reset_index ===


bar,foo,A,B,C
0,one,1,2,3
1,two,4,5,6


RangeIndex(start=0, stop=2, step=1)
Index(['foo', 'A', 'B', 'C'], dtype='object', name='bar')


In [32]:
# pivot()
# - 'index' refers to the column whose unique values will be used as row labels
# - 'columns' refers to the column whose unique values will be used as column labels
# - 'values' refers to the column whose values will be used to fill in the DataFrame
df2 = df.pivot(index='foo', columns='bar', values=['baz', 'zoo'])
print('=== After pivot ===')
display(df2)
print(df2.index)
print(df2.columns)

# Reset index
print('\n=== After reset_index ===')
df2.reset_index(inplace=True)
display(df2)
print(df2.index)
print(df2.columns)

=== After pivot ===


baz       zoo      
bar   A  B  C   A  B  C
foo                    
one   1  2  3   x  y  z
two   4  5  6   q  w  t

Index(['one', 'two'], dtype='object', name='foo')
MultiIndex([('baz', 'A'),
            ('baz', 'B'),
            ('baz', 'C'),
            ('zoo', 'A'),
            ('zoo', 'B'),
            ('zoo', 'C')],
           names=[None, 'bar'])

=== After reset_index ===


foo baz       zoo      
bar        A  B  C   A  B  C
0    one   1  2  3   x  y  z
1    two   4  5  6   q  w  t

RangeIndex(start=0, stop=2, step=1)
MultiIndex([('foo',  ''),
            ('baz', 'A'),
            ('baz', 'B'),
            ('baz', 'C'),
            ('zoo', 'A'),
            ('zoo', 'B'),
            ('zoo', 'C')],
           names=[None, 'bar'])


In [33]:
# pivot() give an error if there are more than one value assigned to the same slot

# A dummy dataframe
df = pd.DataFrame({"foo": ['one', 'one', 'two', 'two'],
                   "bar": ['A', 'A', 'B', 'C'],
                   "baz": [1, 2, 3, 4]})
display(df)

# A ValueError is raised if there are any duplicates
# In this example, there are two values of baz (1 and 2) that are about to be put to the same index of (foo=one, bar=A)
df.pivot(index='foo', columns='bar', values='baz')    # ValueError

,foo,bar,baz
0,one,A,1
1,one,A,2
2,two,B,3
3,two,C,4


ValueError: Index contains duplicate entries, cannot reshape

### **EX2:** Pivot with aggregation using `pivot_table()`


In [34]:
# A dummy dataframe
df = pd.DataFrame({"foo": ['one', 'one', 'two', 'two'],
                   "bar": ['A', 'A', 'B', 'C'],
                   "baz": [1, 2, 3, 4]})

print('=== Original ===')
display(df)

=== Original ===


,foo,bar,baz
0,one,A,1
1,one,A,2
2,two,B,3
3,two,C,4


In [35]:
# pivot_table()
print('\n=== After pivot_table() ===')
df1 = df.pivot_table(index='foo', columns='bar', values='baz')    # default:aggfunc='mean'
display(df1)

# Reset index
print('\n=== After reset_index ===')
df1.reset_index()


=== After pivot_table() ===


bar,A,B,C
foo,,,
one,1.5,NaN,NaN
two,NaN,3.0,4.0



=== After reset_index ===


bar,foo,A,B,C
0,one,1.5,NaN,NaN
1,two,NaN,3.0,4.0


### **EX3:** Pivot table vs. Groupby
- Pivot tables are similar to `groupby()` which is also used for viewing the statistical characteristics of a feature in a dataset.
- Pivot table returns DataFrame whereas `groupby()` returns a DataFrameGroupby object. Hence, conventional DataFrame operations will not work upon the result of `groupby()`.
- There is no clear winner between these two methods. It depends on the situation.

In [36]:
# Data from https://www.kaggle.com/datasets/justinas/nba-players-data
nba_df = pd.read_csv('nba-players.csv')
nba_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12305 entries, 0 to 12304
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         12305 non-null  int64  
 1   player_name        12305 non-null  object 
 2   team_abbreviation  12305 non-null  object 
 3   age                12305 non-null  float64
 4   player_height      12305 non-null  float64
 5   player_weight      12305 non-null  float64
 6   college            10516 non-null  object 
 7   country            12305 non-null  object 
 8   draft_year         12305 non-null  object 
 9   draft_round        12305 non-null  object 
 10  draft_number       12305 non-null  object 
 11  gp                 12305 non-null  int64  
 12  pts                12305 non-null  float64
 13  reb                12305 non-null  float64
 14  ast                12305 non-null  float64
 15  net_rating         12305 non-null  float64
 16  oreb_pct           123

In [37]:
nba_df.head(10)

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Dennis Rodman,CHI,36.0,198.12,99.790240,Southeastern Oklahoma State,USA,1986,2,...,5.7,16.1,3.1,16.1,0.186,0.323,0.100,0.479,0.113,1996-97
1,1,Dwayne Schintzius,LAC,28.0,215.90,117.933920,Florida,USA,1990,1,...,2.3,1.5,0.3,12.3,0.078,0.151,0.175,0.430,0.048,1996-97
2,2,Earl Cureton,TOR,39.0,205.74,95.254320,Detroit Mercy,USA,1979,3,...,0.8,1.0,0.4,-2.1,0.105,0.102,0.103,0.376,0.148,1996-97
3,3,Ed O'Bannon,DAL,24.0,203.20,100.697424,UCLA,USA,1995,1,...,3.7,2.3,0.6,-8.7,0.060,0.149,0.167,0.399,0.077,1996-97
4,4,Ed Pinckney,MIA,34.0,205.74,108.862080,Villanova,USA,1985,1,...,2.4,2.4,0.2,-11.2,0.109,0.179,0.127,0.611,0.040,1996-97
5,5,Eddie Johnson,HOU,38.0,200.66,97.522280,Illinois,USA,1981,2,...,8.2,2.7,1.0,4.1,0.034,0.126,0.220,0.541,0.102,1996-97
6,6,Eddie Jones,LAL,25.0,198.12,86.182480,Temple,USA,1994,1,...,17.2,4.1,3.4,4.1,0.035,0.091,0.209,0.559,0.149,1996-97
7,7,Elden Campbell,LAL,28.0,213.36,113.398000,Clemson,USA,1990,1,...,14.9,8.0,1.6,3.3,0.095,0.183,0.222,0.520,0.087,1996-97
8,8,Eldridge Recasner,ATL,29.0,193.04,86.182480,Washington,USA,1992,Undrafted,...,5.7,1.6,1.3,-0.3,0.036,0.076,0.172,0.539,0.141,1996-97
9,9,Elliot Perry,MIL,28.0,182.88,72.574720,Memphis,USA,1991,2,...,6.9,1.5,3.0,-1.2,0.018,0.081,0.177,0.557,0.262,1996-97


In [38]:
# Choose some columns to work with
df = nba_df[ ['player_name','team_abbreviation','age','pts','season'] ]
df

,player_name,team_abbreviation,age,pts,season
0,Dennis Rodman,CHI,36.0,5.7,1996-97
1,Dwayne Schintzius,LAC,28.0,2.3,1996-97
2,Earl Cureton,TOR,39.0,0.8,1996-97
3,Ed O'Bannon,DAL,24.0,3.7,1996-97
4,Ed Pinckney,MIA,34.0,2.4,1996-97
...,...,...,...,...,...
12300,Markieff Morris,MIA,32.0,7.6,2021-22
12301,Markelle Fultz,ORL,24.0,10.8,2021-22
12302,Marcus Smart,BOS,28.0,12.1,2021-22
12303,Marcus Garrett,MIA,23.0,1.1,2021-22


**Question:** Summarize and display the average player age for each NBA team in each season

In [39]:
# Solution 1 by groupby()
df_groupby = df.groupby(['team_abbreviation','season'])[['age']].agg('mean')
df_groupby

age
team_abbreviation season            
ATL               1996-97  27.529412
                  1997-98  28.250000
                  1998-99  28.500000
                  1999-00  26.692308
                  2000-01  26.187500
...                              ...
WAS               2017-18  27.600000
                  2018-19  26.894737
                  2019-20  25.619048
                  2020-21  26.000000
                  2021-22  25.652174

[772 rows x 1 columns]

In [40]:
# Solution 2 by pivot_table()

# Here, resultant columns are too large to fit the screen
# Also, it is difficult to do season-wise computation on this resultant DataFrame
df_pivot = df.pivot_table( index='team_abbreviation', columns='season', values='age', aggfunc='mean' )
df_pivot

season,1996-97,1997-98,1998-99,1999-00,2000-01,2001-02,2002-03,2003-04,2004-05,2005-06,...,2012-13,2013-14,2014-15,2015-16,2016-17,2017-18,2018-19,2019-20,2020-21,2021-22
team_abbreviation,,,,,,,,,,,,,,,,,,,,,
ATL,27.529412,28.250000,28.500000,26.692308,26.187500,27.000000,27.850000,27.187500,27.529412,23.615385,...,27.187500,27.125000,27.866667,28.266667,28.500000,25.210526,25.111111,25.666667,25.352941,26.142857
BKN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28.058824,29.529412,27.687500,25.941176,26.555556,25.222222,25.470588,26.909091,28.238095,28.470588
BOS,27.647059,25.857143,25.733333,26.642857,26.333333,27.071429,28.666667,26.285714,25.333333,25.071429,...,27.882353,26.250000,25.214286,24.466667,25.733333,25.789474,25.875000,25.000000,24.777778,27.227273
CHA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.411765,25.400000,...,26.666667,26.866667,26.125000,26.250000,25.823529,25.882353,26.411765,24.642857,24.705882,25.117647
CHH,30.230769,29.733333,29.400000,27.941176,28.933333,28.357143,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHI,30.666667,30.600000,28.066667,27.294118,23.625000,25.076923,25.466667,27.187500,26.333333,26.000000,...,28.428571,28.625000,28.571429,27.333333,26.000000,25.157895,24.166667,24.764706,26.411765,25.650000
CLE,26.615385,26.375000,26.937500,27.166667,28.000000,26.533333,23.333333,25.866667,26.687500,26.733333,...,25.437500,24.888889,28.588235,29.058824,30.421053,29.000000,26.142857,24.611111,25.476190,26.045455
DAL,26.941176,26.250000,26.642857,26.823529,26.733333,28.647059,29.750000,28.692308,27.928571,27.687500,...,28.476190,28.733333,29.500000,30.066667,27.263158,27.000000,28.062500,27.222222,26.588235,27.095238
DEN,27.352941,26.400000,25.250000,26.533333,28.666667,27.705882,24.750000,26.666667,28.333333,28.000000,...,25.000000,25.714286,25.666667,25.437500,26.352941,26.705882,25.555556,25.470588,26.555556,27.200000


In [41]:
# Solution 2 by pivot_table()

# Inspect about what causes NA in df_pivot
# List all seasons available for each team
team_list = sorted( df['team_abbreviation'].unique() )
for i,team in enumerate(team_list):
  print( f"===== Team {i+1}/{len(team_list)}: {team} =====" )
  season_list = df.loc[ df['team_abbreviation']==team , 'season' ].unique()
  print( f"#season = {len(season_list)}" )
  print( sorted(season_list), end='\n\n' )

===== Team 1/36: ATL =====
#season = 26
['1996-97', '1997-98', '1998-99', '1999-00', '2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22']

===== Team 2/36: BKN =====
#season = 10
['2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22']

===== Team 3/36: BOS =====
#season = 26
['1996-97', '1997-98', '1998-99', '1999-00', '2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22']

===== Team 4/36: CHA =====
#season = 18
['2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18

## 3.2 Melt or Unpivot (from wide to long)

Codes from https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.melt.html#pandas.DataFrame.melt

In [42]:
# Create the wide dataframe
df = pd.DataFrame({'Name': {0: 'Mary', 1: 'John', 2: 'Bob'},
                   'Age': {0: 20, 1: 30, 2: 15},
                   'Size': {0: 'S', 1: 'L', 2: 'M'}})
df

,Name,Age,Size
0,Mary,20,S
1,John,30,L
2,Bob,15,M


In [43]:
# Turn all columns into rows and list all associated values in a new column value
df.melt()

,variable,value
0,Name,Mary
1,Name,John
2,Name,Bob
3,Age,20
4,Age,30
5,Age,15
6,Size,S
7,Size,L
8,Size,M


In [44]:
df.melt( id_vars=['Name'],      # Column(s) to use as identifier variables (these columns will not be melted)
         value_vars=['Age'] )   # Column(s) to unpivot

,Name,variable,value
0,Mary,Age,20
1,John,Age,30
2,Bob,Age,15


In [45]:
df.melt( id_vars=['Name'],             # Column(s) to use as identifier variables (these columns will not be melted)
         value_vars=['Age','Size'] )   # Column(s) to unpivot

,Name,variable,value
0,Mary,Age,20
1,John,Age,30
2,Bob,Age,15
3,Mary,Size,S
4,John,Size,L
5,Bob,Size,M


# 4. Pandas: Table visualization

In [46]:
# Create a dataset of random numbers
np.random.seed(0)
df = pd.DataFrame(np.random.randn(10,4), columns=['A','B','C','D'])

# Display
df

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


## 4.1 Pandas' dataframe styler

In [47]:
# Be careful, a Styler object is not the same as a DataFrame object
print( f"{type(df.style)=}" )
df.style    # This is a Styler object

type(df.style)=<class 'pandas.io.formats.style.Styler'>


,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [48]:
# Format floating-point precisions
s = df.style.format( formatter={'A':'{:.4f}', 'B':'{:.1f}'} ) # Format the specified columns

# Display
print(f"{type(s)=}")
s

type(s)=<class 'pandas.io.formats.style.Styler'>


,A,B,C,D
0,1.7641,0.4,0.978738,2.240893
1,1.8676,-1.0,0.950088,-0.151357
2,-0.1032,0.4,0.144044,1.454274
3,0.7610,0.1,0.443863,0.333674
4,1.4941,-0.2,0.313068,-0.854096
5,-2.5530,0.7,0.864436,-0.742165
6,2.2698,-1.5,0.045759,-0.187184
7,1.5328,1.5,0.154947,0.378163
8,-0.8878,-2.0,-0.347912,0.156349
9,1.2303,1.2,-0.387327,-0.302303


## 4.2 Unconditional styling

In [51]:
# For *each cell* in DataFrame, style it unconditionally
# Note that setting cell's border styles cannot be done at this cell level

styles = {'background-color':'black', \
              'color':'lawngreen', \
              'font-size':'10px', \
              'text-align':'right'
        }

# Alternative 1: 'set_properties()' is a wrapper to 'map()' when no condition is needed in styling
df.style.set_properties(**styles)

# Alternative 2: create a style function that always returns the same string of CSS styles (unconditional styling)
#style_func = lambda cell_val: ''.join( [ f'{k}:{v};' for k,v in styles.items() ] )
#print(f"{style_func(0)=}")      # Test using the function with any input value
#df.style.map(style_func)   # Apply the style to each cell

style_func(0)='background-color:black;color:lawngreen;font-size:10px;text-align:right;'


,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [52]:
# Style *the entire table* and cells unconditionally

# Create custom styles (th is table header, td is table cell )
custom_styles = [
    dict(selector="table", props=[("border-collapse", "collapse")]),
    dict(selector="th, td", props=[("border", "1px solid red")]),
    dict(selector="td", props=[("background-color", "black"), ("color", "lawngreen")]),
    dict(selector="th", props=[("background-color", "pink"), ("color", "black")])
]

# Apply the custom styles to the DataFrame
df.style.set_table_styles(custom_styles)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [53]:
# Render them to HTML and display the results
print('-------- DataFrame --------')
display( HTML(df.to_html()) )

print('\n-------- Styler --------')
s = df.style.set_table_styles(custom_styles)
display( HTML(s.to_html()) )

-------- DataFrame --------


,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303



-------- Styler --------


,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


## 4.3 Conditional styling

Under-the-hood, the Styler object uses Cascading Style Sheets (CSS) to customise various parameters influencing the display of the DataFrame.
- This is done by passing style functions into the Styler object using `.apply()` or `.map()`.
  - `.map()` : apply a function to the DataFrame <u>element-wise</u>
  - `.apply(...,axis)` : apply a function to the DataFrame <u>row-wise, column-wise, or both</u> (depending on the `axis` parameter)
- <u>The output of style functions should therefore be strings containing CSS attribute-value pair</u>, in the `'attr: value'` format. If nothing is to be applied, the functions should return an empty string `''` or `None`.

### 4.3.1 Style cells that meet a condition

In [55]:
# Style any cell that meets a condition

# Define a style function that takes each cell as an argument
def style_negative_cells( val ):
  # For any cell with negative numbers, change the text to red
  color = 'red' if (val < 0) else ''
  return f'color:{color};'

# Format and style the dataframe
df.style.format( formatter='{:.2f}' ) \
        .map( style_negative_cells )

,A,B,C,D
0,1.76,0.40,0.98,2.24
1,1.87,-0.98,0.95,-0.15
2,-0.10,0.41,0.14,1.45
3,0.76,0.12,0.44,0.33
4,1.49,-0.21,0.31,-0.85
5,-2.55,0.65,0.86,-0.74
6,2.27,-1.45,0.05,-0.19
7,1.53,1.47,0.15,0.38
8,-0.89,-1.98,-0.35,0.16
9,1.23,1.20,-0.39,-0.30


In [56]:
# Generalize the style function with input argument(s)
# Chain many style functions
# Style the selected columns (subset)

# Define a generalized style function (with the 'attr' argument)
def style_negative_cells( val, attr ):
  return attr if (val < 0) else ''

# Format and style the dataframe
# 1. Format all columns (set floating-point precisions)
# 2. Style cells with negative values (change text color)
# 3. Style cells (in the selected columns) whose values in the range (change opacity)
s1= df.style.format( formatter='{:.2f}' ) \
            .map( style_negative_cells, attr='color:red;' ) \
            .map( lambda v: 'opacity:30%;' if (-0.5 < v < 0.5) else None , subset=['A','B'] )
s1

,A,B,C,D
0,1.76,0.40,0.98,2.24
1,1.87,-0.98,0.95,-0.15
2,-0.10,0.41,0.14,1.45
3,0.76,0.12,0.44,0.33
4,1.49,-0.21,0.31,-0.85
5,-2.55,0.65,0.86,-0.74
6,2.27,-1.45,0.05,-0.19
7,1.53,1.47,0.15,0.38
8,-0.89,-1.98,-0.35,0.16
9,1.23,1.20,-0.39,-0.30


### 4.3.2 Style rows/columns that meet a condition

In [57]:
# Style all rows that contain no negative number

# A Style function
def style_allpositive_rows( row, attr ):
  # Choose the color for this one row
  attr = attr if (row >= 0).all() else ''

  # Assign the same color to all columns in this row
  return [ attr for col in row ]

# Apply the function in a row-wise manner
s2 = s1.apply(style_allpositive_rows, attr='background-color:orange;', axis=1)
s2

,A,B,C,D
0,1.76,0.40,0.98,2.24
1,1.87,-0.98,0.95,-0.15
2,-0.10,0.41,0.14,1.45
3,0.76,0.12,0.44,0.33
4,1.49,-0.21,0.31,-0.85
5,-2.55,0.65,0.86,-0.74
6,2.27,-1.45,0.05,-0.19
7,1.53,1.47,0.15,0.38
8,-0.89,-1.98,-0.35,0.16
9,1.23,1.20,-0.39,-0.30


In [58]:
# Highlight the maximum value(s) regarding each column

# A Style function
def style_max(val, attr=''):
  # If a value equals the maximum (ignore NA), set it to 'attr'. Otherwise, set it to ''.
  return np.where(val == np.nanmax(val.values), attr, '')

# Use blue cells to highlight the maximum of the column
df.style.format( formatter='{:.2f}' ) \
        .map( style_negative_cells, attr='color:red;' ) \
        .apply(style_max, attr='color:white;background-color:darkblue', axis=0)

,A,B,C,D
0,1.76,0.40,0.98,2.24
1,1.87,-0.98,0.95,-0.15
2,-0.10,0.41,0.14,1.45
3,0.76,0.12,0.44,0.33
4,1.49,-0.21,0.31,-0.85
5,-2.55,0.65,0.86,-0.74
6,2.27,-1.45,0.05,-0.19
7,1.53,1.47,0.15,0.38
8,-0.89,-1.98,-0.35,0.16
9,1.23,1.20,-0.39,-0.30


In [61]:
# Highlight the maximum value(s) regarding each row

# Use pink cells to highlight the maximum of the row
df.style.format( formatter='{:.2f}' ) \
        .map( style_negative_cells, attr='color:red;' ) \
        .apply(style_max, attr='color:white;background-color:pink;color:black', axis=1)

,A,B,C,D
0,1.76,0.40,0.98,2.24
1,1.87,-0.98,0.95,-0.15
2,-0.10,0.41,0.14,1.45
3,0.76,0.12,0.44,0.33
4,1.49,-0.21,0.31,-0.85
5,-2.55,0.65,0.86,-0.74
6,2.27,-1.45,0.05,-0.19
7,1.53,1.47,0.15,0.38
8,-0.89,-1.98,-0.35,0.16
9,1.23,1.20,-0.39,-0.30


In [62]:
# Style the maximum value of the whole dataframe
# Set the table caption

# Use purple cells to highlight the maximum of the row
df.style.set_caption('Style the maximum of the  dataset') \
        .format( formatter='{:.2f}' ) \
        .map( style_negative_cells, attr='color:red;' ) \
        .apply(style_max, attr='color:white;background-color:purple;color:white', axis=None)

,A,B,C,D
0,1.76,0.40,0.98,2.24
1,1.87,-0.98,0.95,-0.15
2,-0.10,0.41,0.14,1.45
3,0.76,0.12,0.44,0.33
4,1.49,-0.21,0.31,-0.85
5,-2.55,0.65,0.86,-0.74
6,2.27,-1.45,0.05,-0.19
7,1.53,1.47,0.15,0.38
8,-0.89,-1.98,-0.35,0.16
9,1.23,1.20,-0.39,-0.30


## 4.4 Built-in styles

https://pandas.pydata.org/docs/user_guide/style.html#Builtin-Styles
- `.highlight_null()`
- `.highlight_min()`
- `.highlight_max()`
- `.highlight_between()`
- `.highlight_quantile()`
- `.background_gradient()`
- `.text_gradient()`
- `.bar()`

In [64]:
# The highlight_min and highlight_max built-in styles

# Style max of each column with blue background
# Style min of dataframe with green background
df.style.set_caption('Blue:max of column | Green:min of dataframe') \
        .format( formatter='{:.2f}' ) \
        .map( style_negative_cells, attr='color:red;' ) \
        .highlight_max(axis=0, props='color:white;background-color:darkblue') \
        .highlight_min(axis=None, props='color:white;background-color:darkgreen')

,A,B,C,D
0,1.76,0.40,0.98,2.24
1,1.87,-0.98,0.95,-0.15
2,-0.10,0.41,0.14,1.45
3,0.76,0.12,0.44,0.33
4,1.49,-0.21,0.31,-0.85
5,-2.55,0.65,0.86,-0.74
6,2.27,-1.45,0.05,-0.19
7,1.53,1.47,0.15,0.38
8,-0.89,-1.98,-0.35,0.16
9,1.23,1.20,-0.39,-0.30


In [65]:
# The background_gradient built-in style

# Make a sequential palette that blends from light to 'green'
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)

# Style and display
df.style.background_gradient(cmap=cm)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [66]:
# The bar built-in style: include a bar chart in the DataFrame

# One color for all values
df.style.bar(subset=['A', 'B'], color='#d65f5f')

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [67]:
# The bar built-in style: include a bar chart in the DataFrame

# One color for positive, one color for negative
color_negative , color_positive = 'red', 'green'
df.style.bar(subset=['A', 'B'], color=[color_negative, color_positive])

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [68]:
# The bar built-in style: include a bar chart in the DataFrame
# API: https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.bar.html
# Example: https://pandas.pydata.org/docs/user_guide/style.html#Bar-charts

df.style.bar( align=0,               # Bar alignment
              vmin=-2.5, vmax=2.5,   # Range of the bar value
              cmap="bwr",            # Use colormap instead of specifying 'color=' by ourselves
              height=70, width=100,  # (default:100) Percentage of the bar in the cell
              props="width:120px; border-right: 1px solid black;"  # Cell's attribute
            )

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


# PRACTICE

1. (3 point) Display pokemons by pokedex number, English name, and the number of its possible moves. Sort pokemons by the number of possible moves (from largest to smallest). Resultant dataframe expected by this question is shown in `pandas4_question1_answer.csv`.
  - The number of possible moves regarding a pokemon is considered from their type1 and type2.
  - Be careful not to over count the possible moves when type1 and type2 are the same.
  - Be careful not to count the same move more than once.